In [ ]:
application "tropical";

# A larger example
The previous examples had a relatively small dimension, so we could not study the effects of having different non-trivial sedentarities.

In [ ]:
$vert = new Matrix(
[
[1,0,0,0],
[1,0,0,1],
[1,0,0,2],
[1,0,0,3],
[1,0,1,0],
[1,0,1,1],
[1,0,1,2],
[1,0,2,0],
[1,0,2,1],
[1,0,3,0],
[1,1,0,0],
[1,1,0,1],
[1,1,0,2],
[1,1,1,0],
[1,1,1,1],
[1,1,2,0],
[1,2,0,0],
[1,2,0,1],
[1,2,1,0],
[1,3,0,0]
]);
$vert = new Matrix<Int>(map(new Vector<Int>(3-$_->[1]-$_->[2]-$_->[3],$_->[1],$_->[2],$_->[3]), @$vert));
print rows_labeled($vert),"\n";
$tr = new Array<Set<Int>>([[0,1,4,10],[10,11,13,16],[14,15,16,17],[15,16,17,18],[16,17,18,19],[2,3,6,12],[2,6,8,12],[11,12,14,17],[8,9,11,14],[9,11,14,15],[9,11,13,15],[11,13,15,16],[7,10,11,13],[7,9,11,13],[1,4,7,10],[1,2,5,11],[5,8,9,11],[5,7,9,11],[2,5,8,11],[11,14,16,17],[11,14,15,16],[5,7,10,11],[1,5,10,11],[1,5,7,10],[2,11,12,14],[2,8,12,14],[2,8,11,14]]
);
$S = new fan::SubdivisionOfPoints(POINTS=>ones_vector<Int> | $vert, MAXIMAL_CELLS=>$tr);
print rows_labeled($vert),"\n";

In [ ]:
print $S->REGULAR,"\n";
$v = primitive($S->WEIGHTS);
$trop = new Hypersurface<Min>(MONOMIALS=>$vert, COEFFICIENTS=>$v);
print $v;

In [ ]:
print $trop->VERTICES;

In [ ]:
print $trop->N_VERTICES;

In [ ]:
print rows_labeled($trop->COMPACTIFICATION->DECORATION);

In [ ]:
$p = $trop->PATCHWORK(SIGNS=>ones_vector<Int>(20));

In [ ]:
# print transpose($p->REAL_FACETS);
print $trop->VERTICES;

The rays of the tropical hypersurface should be the rays of the normal fan of the simplex. However we do not know where and in which order they appear. The following method will label the rays accordingly. It is just an auxiliary method for building the right projection maps to the boundary.

In [ ]:
sub relabel {
    my($vert) = @_;
    my $d = $vert->cols();
    my $result = new Array<Int>($vert->rows());
    for(my $i=0; $i<$vert->rows(); $i++){
        if($vert->row($i)->[0] == 0){
            my $v = $vert->row($i)->slice(sequence(2,$d-2));
            for(my $j=0; $j<$d-2; $j++){
                if($v == -unit_vector($d-2,$j) || $v == unit_vector($d-2,$j)){
                    # print "Found: $i $j\n";
                    $result->[$i] = $j;
                    last;
                }
            }
            if($v == ones_vector($d-2) || $v == -ones_vector($d-2)){
                $result->[$i] = -1;
            }
        }
    }
    return $result;
}

In [ ]:
print relabel($trop->VERTICES);

There are two main cases for the sedentarity of a face: It can contain the all ones vector or not. For both cases we assemble projection and lifting matrices. The lifting is needed as we can build projections between faces with non-trivial sedentarity by going via the lifting to the parent face.

In [ ]:
sub get_gf2_projections {
    my($h) = @_;
    my $relabel = relabel($h->VERTICES);
    my $d = $h->VERTICES->cols() - 2;
    my $result = new Map<Set<Int>, Pair<Matrix<GF2>, Matrix<GF2>>>();
    my $comp = $h->COMPACTIFICATION;
    my $decor = $comp->DECORATION;
    for(my $node = entire(nodes($comp->ADJACENCY)); $node; ++$node){
        my $psed = $decor->[$$node]->sedentarity;
        print $$node,": $psed ";
        my $sed = new Set<Int>();
        foreach my $entry (@$psed){
            $sed += $relabel->[$entry];
        }
        my ($projection, $lifting);
        if($sed->contains(-1)){
            my $first = 0;
            while($sed->contains($first)){
                $first++;
            }
            $first < $h->N_RAYS or die "Could not find non-contained element";
            $projection = new Matrix<GF2>(unit_matrix<GF2>($d));
            $projection->col($first) = ones_vector<GF2>($d);
            $lifting = inv($projection);
            $sed -= -1;
            $sed += $first;
            $projection = $projection->minor(~$sed, All);
            $lifting = $lifting->minor(All, ~$sed);
        } else {
            $projection = unit_matrix<GF2>($d);
            $lifting = unit_matrix<GF2>($d);
            $projection = $projection->minor(~$sed, All);
            $lifting = $lifting->minor(All, ~$sed);
        }
        my $prod = $projection * $lifting;
        my $check = $prod == unit_matrix<GF2>($projection->rows);
        my $vert = $h->VERTICES->minor($psed, sequence(2,$d));
        $prod = $projection * transpose(new Matrix<GF2>(convert_to<Integer>($vert)));
        $check &= $prod == zero_matrix<GF2>($prod->rows, $prod->cols);
        print "Check: $check\n";
        if($check == 0){
            print "Proj:\n",$projection,"\n";
            print "Vert:\n",$vert,"\n";
            print "Prod:\n",$prod,"\n";
        }
        $result->{$psed} = new Pair<Matrix<GF2>, Matrix<GF2>>($projection, $lifting);
    }
    return $result;
}

In [ ]:
$projlift = get_gf2_projections($trop);

In [ ]:
print $trop->PATCHWORK->REAL_FACETS->rows;

In [ ]:
print $trop->PATCHWORK->REAL_FACETS;

The following method converts a number to a vector in GF2 corresponding to it's binary representation.

In [ ]:
sub number_to_gf2vector {
    my($length, $n) = @_;
    my $result = new Vector<GF2>($length);
    for(my $i=0; $i<$length; $i++){
        $result->[$i] = $n % 2;
        $n -= $n%2;
        $n /= 2;
    }
    $n == 0 or die "Incomplete conversion";
    return $result;
}

In [ ]:
print number_to_gf2vector(4,5);

Using this method we can collect the elements of the affine GF2-spaces attached to every facet.

In [ ]:
$comp = $trop->COMPACTIFICATION;
$decor = $comp->DECORATION;
$facets = $trop->MAXIMAL_POLYTOPES;
$real_facets = $trop->PATCHWORK->REAL_FACETS;
$nm = new NodeMap<Directed, HashSet<Vector<GF2>>>($comp->ADJACENCY);
foreach my $node (@{$comp->nodes_of_rank($trop->DIM+1)}){
    my $real = $decor->[$node]->realisation;
    # Only works since the thing is pure
    my $facetno = -1;
    for(my $i=0; $i<$facets->rows; $i++){
        if($facets->[$i] == $real){
            $facetno = $i;
            last;
        }
    }
    print $node," ",$real," ",$facetno,": ";

    for(my $i = 0; $i<$real_facets->rows; $i++){
        if($real_facets->[$i]->contains($facetno)){
            print $i,",";
            $nm->[$node] += number_to_gf2vector($trop->DIM+1, $i);
        }
    }
    print "\n";
    
}

Now we can propagate these spaces downwards in the Hasse diagram. We need to take boundary effects given by the sedentarity into account. For an edge corresponding to a face relation of two faces with non-trivial sedentarity we proceed by first lifting and then projecting, which corresponds to the canonical projection between spaces.

In [ ]:
for(my $i = $trop->DIM; $i>0; $i--){
    foreach my $node (@{$comp->nodes_of_rank($i)}){
        my $sed = $decor->[$node]->sedentarity;
        foreach my $parent (@{$comp->ADJACENCY->out_adjacent_nodes($node)}){
            my $psed = $decor->[$parent]->sedentarity;
            my $lift = $projlift->{$psed}->second;
            my $proj = $projlift->{$sed}->first;
            my $transport = $proj * $lift;
            foreach my $v (@{$nm->[$parent]}){
                if($transport->cols != $v->dim){
                    print $transport,"\n",$v,"\n";
                    print "Node: $node, parent: $parent\n";
                    die "Wrong num of cols.";
                }
                $nm->[$node] += $transport * $v;
            }
            # $nm->[$node] += new HashSet<Vector<GF2>>(map($transport * $_, @{$nm->[$parent]}));
        }
    }
}

In [ ]:
print $comp->TOP_NODE," ",$comp->BOTTOM_NODE,"\n";
print $decor->[0],"\n";
print $nm;

Now every node in the Hasse diagram is equipped with the index set of a basis of the corresponding GF2-vectorspace. The canonical maps on the index sets give rise to maps of the vector spaces as well. We collect these in an `EdgeMap` mapping every edge of the Hasse diagram to a matrix. This is the same as a cellular sheaf.

In [ ]:
$em = new EdgeMap<Directed, Matrix<GF2>>($comp->ADJACENCY);

When building the matrix for one edge we again need to take the sedentarity into account. We do this in the same way as before.

In [ ]:
sub make_matrix {
    my($edge, $comp, $bases) = @_;
    my $source = $edge->to_node;
    my $target = $edge->from_node;
    my $decor = $comp->DECORATION;
    my $sed = $decor->[$target]->sedentarity;
    my $psed = $decor->[$source]->sedentarity;
    # print "$sed - $psed\n";
    my $lift = $projlift->{$psed}->second;
    my $proj = $projlift->{$sed}->first;
    my $transport = $proj * $lift;
    # print "Transport:\n",$transport;
    # print "S: ",$source,"T: ",$target,"\n";
    my $result = new Matrix<GF2>($bases->[$target]->size, $bases->[$source]->size);
    my $i = 0;
    foreach my $sv (@{$bases->[$source]}) {
        # print $tv,"\n";
        my $tsv = $transport * $sv;
        my $j = 0;
        foreach my $tv (@{$bases->[$target]}) {
            if($tsv == $tv){
                $result->elem($j, $i) = 1
            }
            $j++;
        }
        $i++;
    }
    
    return $result;
}

In [ ]:
for (my $e=entire(edges($comp->ADJACENCY)); $e; ++$e) {
    $em->[$$e] = make_matrix($e, $comp, $nm);
}

For the top node we just create artificial zero matrices of the right dimensions.

In [ ]:
$top = $comp->TOP_NODE;
foreach my $node (@{$comp->ADJACENCY->in_adjacent_nodes($top)}){
    $em->edge($node, $top) = zero_matrix<GF2>($nm->[$node]->size(),0);
}

Now we can assemble the associated chain complex from the cellular sheaf.

In [ ]:
$chain = fan::build_full_chain($comp, $comp->ORIENTATIONS, $em, true);

In [ ]:
print $chain->type->full_name;

In [ ]:
print topaz::betti_numbers<GF2>($chain);